In [ ]:
### Mutual Information estimators comparisson for Gaussian distribution

In [ ]:
"""This script calculates the following MI quantities for four different estimators 
(Shannon, Miller-Madow, KL, KSG) for random variables drawn from a Gaussian 
distribution, with different correaltions and different data size:
1) Two-way MI, MI2
2) Total Correlation, TC
3) Three-way MI, MI3
4) Interaction Information, II
5) Conditional MI, CMI

(*) this is repeated 100 times (replicates)
"""

In [ ]:
import sys
import os
import numpy as np
import time
import scipy.io

"""# Add the folder path of the scripts & modules to the sys.path list
path_to_code = os.path.expanduser('~/Dropbox/Roberts/CODE_rsync/')
sys.path.append(path_to_code)"""

### Information calculation with units of [nats]
import Mutual_Info_based_binning_module as MI_FB_mod
import Mutual_Info_KNN_nats_module as MI_KNN_mod
import Building_MI_matrices as Building_MI_matrices_mod

In [ ]:
### Save path to where data will be stored
#path_to_data = os.path.expanduser('~/DATA/MI_comparison_FB_vs_KNN/')
project_directory = os.getcwd().split('CODE')[0]
path_to_data = project_directory + '/DATA/MI_comparison_FB_vs_KNN/'

In [ ]:
### global constants initialization
corr_list = [0.3, 0.6, 0.9]
Ntot_list = [100,1000,10000]
#Ntot_list = [100,250] # debug
iterations = 100
#iterations = 3 # debug
max_num_of_bins = 101
k_max = 10 # k-NN

In [ ]:
### FB in 2D

### local constants initialization
np.random.seed(13)  ### CHANGE ME IF NEEDED
mi_est = "Shannon"

MI2_bins_4d_array = np.zeros((len(corr_list),len(Ntot_list),max_num_of_bins-2,iterations),dtype=float) 

for iteration in range(iterations):
    
    for c,correlation in enumerate(corr_list):
        covs = [[1, correlation], [correlation, 1]]
        m = np.random.multivariate_normal([0,0], covs, max(Ntot_list)).T

        for n,Ntot in enumerate(Ntot_list):
            for num_of_bins in range(2,max_num_of_bins):
                MI2_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Two_way_info(m[0][:Ntot],m[1][:Ntot],num_of_bins,mi_est)

print("MI2 bins 4d array shape",MI2_bins_4d_array.shape)

dict_name= 'MI2_bins_4d_array'
matfile = path_to_data + dict_name + '.mat'
scipy.io.savemat(matfile, mdict={dict_name: MI2_bins_4d_array})

In [ ]:
### Miller-Madow FB in 2D

# local constants initialization
np.random.seed(13)  ### CHANGE ME IF NEEDED
mi_est = "Miller-Madow"

MI2_bins_4d_array = np.zeros((len(corr_list),len(Ntot_list),max_num_of_bins-2,iterations),dtype=float) 

for iteration in range(iterations):
    
    for c,correlation in enumerate(corr_list):
        covs = [[1, correlation], [correlation, 1]]
        m = np.random.multivariate_normal([0,0], covs, max(Ntot_list)).T

        for n,Ntot in enumerate(Ntot_list):
            for num_of_bins in range(2,max_num_of_bins):
                MI2_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Two_way_info(m[0][:Ntot],m[1][:Ntot],num_of_bins,mi_est)

print("MI2 bins 4d array shape",MI2_bins_4d_array.shape)

dict_name= 'MI2_MM_bins_4d_array'
matfile = path_to_data + dict_name + '.mat'
scipy.io.savemat(matfile, mdict={dict_name: MI2_bins_4d_array})

In [ ]:
### 3D with 100 iterations

# local constants initialization
np.random.seed(13)  ### CHANGE ME IF NEEDED
mi_est = "Shannon"
TC_bins_4d_array = np.zeros((len(corr_list),len(Ntot_list),max_num_of_bins-2,iterations),dtype=float) 
II_bins_4d_array = np.zeros((len(corr_list),len(Ntot_list),max_num_of_bins-2,iterations),dtype=float) 
CMI_bins_4d_array = np.zeros((len(corr_list),len(Ntot_list),max_num_of_bins-2,iterations),dtype=float) 
MI3_bins_4d_array = np.zeros((len(corr_list),len(Ntot_list),max_num_of_bins-2,iterations),dtype=float) 
t1 = time.time()

for iteration in range(iterations):
    for c,correlation in enumerate(corr_list):
        covs = [[1, correlation, correlation], [correlation, 1, correlation], [correlation, correlation, 1]]
        m = np.random.multivariate_normal([0,0,0], covs, max(Ntot_list)).T

        for n,Ntot in enumerate(Ntot_list):

            for num_of_bins in range(2,max_num_of_bins):
                ### Calculating MI quantities - slow if using to calculate more than one quantity
                #TC_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Total_Corr(m[0][:Ntot],m[1][:Ntot],m[2][:Ntot],num_of_bins)
                #II_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Inter_Info(m[0][:Ntot],m[1][:Ntot],m[2][:Ntot],num_of_bins)
                #CMI_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Conditional_mutual_info(m[0][:Ntot],m[1][:Ntot],m[2][:Ntot],num_of_bins)
                #MI3_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Three_way_info(m[0][:Ntot],m[1][:Ntot],m[2][:Ntot],num_of_bins)
                
                ### Calculating from entropies - this is faster when calculating all 3d MI quantites
                Ex = MI_FB_mod.Entropy(m[0][:Ntot],num_of_bins,mi_est)
                Ey = MI_FB_mod.Entropy(m[1][:Ntot],num_of_bins,mi_est)
                Ez = MI_FB_mod.Entropy(m[2][:Ntot],num_of_bins,mi_est)
                Exy = MI_FB_mod.Entropy2var(m[0][:Ntot],m[1][:Ntot],num_of_bins,mi_est)
                Exz = MI_FB_mod.Entropy2var(m[0][:Ntot],m[2][:Ntot],num_of_bins,mi_est)
                Eyz = MI_FB_mod.Entropy2var(m[1][:Ntot],m[2][:Ntot],num_of_bins,mi_est)
                Exyz = MI_FB_mod.Entropy3var(m[0][:Ntot],m[1][:Ntot],m[2][:Ntot],num_of_bins,mi_est)

                TC_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Total_Corr_from_entropy(Ex,Ey,Ez,Exyz)
                II_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Inter_Info_from_entropy(Ex,Ey,Ez,Exy,Exz,Eyz,Exyz)
                CMI_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Conditional_mutual_info_from_entropy(Exz,Eyz,Exyz,Ez)
                MI3_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Three_way_info_from_entropy(Ez,Exy,Exyz)

t2 = time.time()
print("Run time = %.2f [sec]" %(t2-t1))

dict_name_list = ['TC_bins_4d_array','II_bins_4d_array','CMI_bins_4d_array','MI3_bins_4d_array']
data_4d_array_list = [TC_bins_4d_array,II_bins_4d_array,CMI_bins_4d_array,MI3_bins_4d_array]

for counter,dict_name in enumerate(dict_name_list):
    matfile = path_to_data + dict_name + '.mat'
    scipy.io.savemat(matfile, mdict={dict_name: data_4d_array_list[counter]})

In [ ]:
### Miller-Madow FB 3D calc with 100 iterations

# local constants initialization
np.random.seed(13)  ### CHANGE ME IF NEEDED
mi_est = "Miller-Madow"
TC_bins_4d_array = np.zeros((len(corr_list),len(Ntot_list),max_num_of_bins-2,iterations),dtype=float) 
II_bins_4d_array = np.zeros((len(corr_list),len(Ntot_list),max_num_of_bins-2,iterations),dtype=float) 
CMI_bins_4d_array = np.zeros((len(corr_list),len(Ntot_list),max_num_of_bins-2,iterations),dtype=float) 
MI3_bins_4d_array = np.zeros((len(corr_list),len(Ntot_list),max_num_of_bins-2,iterations),dtype=float) 
t1 = time.time()

for iteration in range(iterations):
    for c,correlation in enumerate(corr_list):
        covs = [[1, correlation, correlation], [correlation, 1, correlation], [correlation, correlation, 1]]
        m = np.random.multivariate_normal([0,0,0], covs, max(Ntot_list)).T

        for n,Ntot in enumerate(Ntot_list):

            for num_of_bins in range(2,max_num_of_bins):
                ### Calculating MI quantities - slow if using to calculate more than one quantity
                #TC_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Total_Corr(m[0][:Ntot],m[1][:Ntot],m[2][:Ntot],num_of_bins,mi_est)
                #II_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Inter_Info(m[0][:Ntot],m[1][:Ntot],m[2][:Ntot],num_of_bins,mi_est)
                #CMI_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Conditional_mutual_info(m[0][:Ntot],m[1][:Ntot],m[2][:Ntot],num_of_bins,mi_est)
                #MI3_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Three_way_info(m[0][:Ntot],m[1][:Ntot],m[2][:Ntot],num_of_bins,mi_est)
                
                ### Calculating from entropies - this is faster when calculating all 3d MI quantites
                Ex = MI_FB_mod.Entropy(m[0][:Ntot],num_of_bins,mi_est)
                Ey = MI_FB_mod.Entropy(m[1][:Ntot],num_of_bins,mi_est)
                Ez = MI_FB_mod.Entropy(m[2][:Ntot],num_of_bins,mi_est)
                Exy = MI_FB_mod.Entropy2var(m[0][:Ntot],m[1][:Ntot],num_of_bins,mi_est)
                Exz = MI_FB_mod.Entropy2var(m[0][:Ntot],m[2][:Ntot],num_of_bins,mi_est)
                Eyz = MI_FB_mod.Entropy2var(m[1][:Ntot],m[2][:Ntot],num_of_bins,mi_est)
                Exyz = MI_FB_mod.Entropy3var(m[0][:Ntot],m[1][:Ntot],m[2][:Ntot],num_of_bins,mi_est)

                TC_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Total_Corr_from_entropy(Ex,Ey,Ez,Exyz)
                II_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Inter_Info_from_entropy(Ex,Ey,Ez,Exy,Exz,Eyz,Exyz)
                CMI_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Conditional_mutual_info_from_entropy(Exz,Eyz,Exyz,Ez)
                MI3_bins_4d_array[c][n][num_of_bins-2][iteration] = MI_FB_mod.Three_way_info_from_entropy(Ez,Exy,Exyz)

t2 = time.time()
print("Run time = %.2f [sec]" %(t2-t1))

dict_name_list = ['TC_MM_bins_4d_array','II_MM_bins_4d_array','CMI_MM_bins_4d_array','MI3_MM_bins_4d_array']
data_4d_array_list = [TC_bins_4d_array,II_bins_4d_array,CMI_bins_4d_array,MI3_bins_4d_array]

for counter,dict_name in enumerate(dict_name_list):
    matfile = path_to_data + dict_name + '.mat'
    scipy.io.savemat(matfile, mdict={dict_name: data_4d_array_list[counter]})

In [ ]:
### 2D KNN multiple iterations

# local constants initialization
np.random.seed(13)  ### CHANGE ME IF NEEDED
MI2_knn_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
t1 = time.time()

for iteration in range(iterations):
    
    for c,correlation in enumerate(corr_list):
        covs = [[1, correlation], [correlation, 1]]
        m = np.random.multivariate_normal([0,0], covs, max(Ntot_list)).T

        for n,Ntot in enumerate(Ntot_list):
            MI2_knn_4d_array[c,n,:,iteration] = MI_KNN_mod.MI_KNN_KDtree_algo(m[0][:Ntot],m[1][:Ntot],k_max)

t2 = time.time()
print("Run time = %.2f [sec]" %(t2-t1))

dict_name= 'MI2_knn_KSG_4d_array'
matfile = path_to_data + dict_name + '.mat'
scipy.io.savemat(matfile, mdict={dict_name: MI2_knn_4d_array})

In [ ]:
### 3D KNN

# local constants initialization
np.random.seed(13)  ### CHANGE ME IF NEEDED
MI2xy_knn_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
MI2xz_knn_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
MI2yz_knn_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
TC_knn_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
II_knn_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
CMI_knn_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
MI3_knn_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
t1 = time.time()

for iteration in range(iterations):
    
    for c,correlation in enumerate(corr_list):
        covs = [[1, correlation, correlation], [correlation, 1, correlation], [correlation, correlation, 1]]
        m = np.random.multivariate_normal([0,0,0], covs, max(Ntot_list)).T

        for n,Ntot in enumerate(Ntot_list):
            MI2xy_knn_4d_array[c,n,:,iteration] = MI_KNN_mod.MI_KNN_KDtree_algo(m[0][:Ntot],m[1][:Ntot],k_max)
            MI2xz_knn_4d_array[c,n,:,iteration] = MI_KNN_mod.MI_KNN_KDtree_algo(m[0][:Ntot],m[1][:Ntot],k_max)
            MI2yz_knn_4d_array[c,n,:,iteration] = MI_KNN_mod.MI_KNN_KDtree_algo(m[0][:Ntot],m[1][:Ntot],k_max)
            TC_knn_4d_array[c,n,:,iteration] = MI_KNN_mod.TC_KNN_KDtree_algo(m[0][:Ntot],m[1][:Ntot],m[2][:Ntot],k_max)
            II_knn_4d_array[c,n,:,iteration] = TC_knn_4d_array[c,n,:,iteration] - MI2xy_knn_4d_array[c,n,:,iteration] - MI2xz_knn_4d_array[c,n,:,iteration] - MI2yz_knn_4d_array[c,n,:,iteration]
            CMI_knn_4d_array[c,n,:,iteration] = TC_knn_4d_array[c,n,:,iteration] - MI2xz_knn_4d_array[c,n,:,iteration] - MI2yz_knn_4d_array[c,n,:,iteration]
            MI3_knn_4d_array[c,n,:,iteration] = TC_knn_4d_array[c,n,:,iteration] - MI2xy_knn_4d_array[c,n,:,iteration]


t2 = time.time()
print("Run time = %.2f [sec]" %(t2-t1))

dict_name_list = ['MI2xy_knn_KSG_4d_array','MI2xz_knn_KSG_4d_array','MI2yz_knn_KSG_4d_array','TC_knn_KSG_4d_array','II_knn_KSG_4d_array','CMI_knn_KSG_4d_array','MI3_knn_KSG_4d_array']
data_4d_array_list = [MI2xy_knn_4d_array,MI2xz_knn_4d_array,MI2yz_knn_4d_array,TC_knn_4d_array,II_knn_4d_array,CMI_knn_4d_array,MI3_knn_4d_array]

for counter,dict_name in enumerate(dict_name_list):
    matfile = path_to_data + dict_name + '.mat'
    scipy.io.savemat(matfile, mdict={dict_name: data_4d_array_list[counter]})

In [ ]:
### Adding KL-KNN to comparisson

# local constants initialization
np.random.seed(13)  ### CHANGE ME IF NEEDED
MI2_knn_KL_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
t1 = time.time()

for iteration in range(iterations):
    
    for c,correlation in enumerate(corr_list):
        covs = [[1, correlation], [correlation, 1]]
        m = np.random.multivariate_normal([0,0], covs, max(Ntot_list)).T

        for n,Ntot in enumerate(Ntot_list):
            Ex = MI_KNN_mod.Entropy_KNN_KDtree_algo(m[0][:Ntot],k_max)
            Ey = MI_KNN_mod.Entropy_KNN_KDtree_algo(m[1][:Ntot],k_max)
            Exy = MI_KNN_mod.Entropy2D_KNN_KDtree_algo(m[0][:Ntot],m[1][:Ntot],k_max)

            MI2_knn_KL_4d_array[c,n,:,iteration] = MI_KNN_mod.Two_way_info_from_entropy(Ex,Ey,Exy)

t2 = time.time()
print("Run time = %.3f [sec]" %(t2-t1))

dict_name= 'MI2_knn_KL_4d_array'
matfile = path_to_data + dict_name + '.mat'
scipy.io.savemat(matfile, mdict={dict_name: MI2_knn_KL_4d_array})

In [ ]:
### Adding KL-KNN to comparisson

# local constants initialization
np.random.seed(13)  ### CHANGE ME IF NEEDED
MI2xy_knn_KL_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
MI2xz_knn_KL_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
MI2yz_knn_KL_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
TC_knn_KL_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
II_knn_KL_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
CMI_knn_KL_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
MI3_knn_KL_4d_array = np.zeros((len(corr_list),len(Ntot_list),k_max,iterations),dtype=float) 
t1 = time.time()

for iteration in range(iterations):
    
    for c,correlation in enumerate(corr_list):
        covs = [[1, correlation, correlation], [correlation, 1, correlation], [correlation, correlation, 1]]
        m = np.random.multivariate_normal([0,0,0], covs, max(Ntot_list)).T

        for n,Ntot in enumerate(Ntot_list):
            Ex = MI_KNN_mod.Entropy_KNN_KDtree_algo(m[0][:Ntot],k_max)
            Ey = MI_KNN_mod.Entropy_KNN_KDtree_algo(m[1][:Ntot],k_max)
            Ez = MI_KNN_mod.Entropy_KNN_KDtree_algo(m[2][:Ntot],k_max)
            Exy = MI_KNN_mod.Entropy2D_KNN_KDtree_algo(m[0][:Ntot],m[1][:Ntot],k_max)
            Exz = MI_KNN_mod.Entropy2D_KNN_KDtree_algo(m[0][:Ntot],m[2][:Ntot],k_max)
            Eyz = MI_KNN_mod.Entropy2D_KNN_KDtree_algo(m[1][:Ntot],m[2][:Ntot],k_max)
            Exyz = MI_KNN_mod.Entropy3D_KNN_KDtree_algo(m[0][:Ntot],m[1][:Ntot],m[2][:Ntot],k_max)

            MI2xy_knn_KL_4d_array[c,n,:,iteration] = MI_KNN_mod.Two_way_info_from_entropy(Ex,Ey,Exy)
            MI2xz_knn_KL_4d_array[c,n,:,iteration] = MI_KNN_mod.Two_way_info_from_entropy(Ex,Ez,Exz)
            MI2yz_knn_KL_4d_array[c,n,:,iteration] = MI_KNN_mod.Two_way_info_from_entropy(Ey,Ez,Eyz)
            TC_knn_KL_4d_array[c,n,:,iteration] = MI_KNN_mod.Total_Corr_from_entropy(Ex,Ey,Ez,Exyz)
            II_knn_KL_4d_array[c,n,:,iteration] = MI_KNN_mod.Inter_Info_from_entropy(Ex,Ey,Ez,Exy,Exz,Eyz,Exyz)
            CMI_knn_KL_4d_array[c,n,:,iteration] = MI_KNN_mod.Conditional_mutual_info_from_entropy(Exz,Eyz,Exyz,Ez)
            MI3_knn_KL_4d_array[c,n,:,iteration] = MI_KNN_mod.Three_way_info_from_entropy(Ez,Exy,Exyz)


t2 = time.time()
print("Run time = %.3f [sec]" %(t2-t1))

dict_name_list = ['TC_knn_KL_4d_array','II_knn_KL_4d_array','CMI_knn_KL_4d_array','MI3_knn_KL_4d_array']
data_4d_array_list = [TC_knn_KL_4d_array,II_knn_KL_4d_array,CMI_knn_KL_4d_array,MI3_knn_KL_4d_array]

for counter,dict_name in enumerate(dict_name_list):
    matfile = path_to_data + dict_name + '.mat'
    scipy.io.savemat(matfile, mdict={dict_name: data_4d_array_list[counter]})